In [1]:
import pandas as pd
from transformers import pipeline 
import numpy as np
import json

In [2]:
def load_json(path):
    data = []
    with open(path, "r") as f:
        for line in f:
            data.append(json.loads(line))
    return pd.DataFrame.from_records(data)

In [3]:
dataset = load_json('./datasets/climate_id_2.jsonl')

MemoryError: 

In [ ]:
print(len(dataset))

184775


In [ ]:
## Location Names====================================================
state_codes = [ 'AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY']

state_code_to_name = {
    'AK': 'Alaska',
    'AL': 'Alabama',
    'AR': 'Arkansas',
    'AZ': 'Arizona',
    'CA': 'California',
    'CO': 'Colorado',
    'CT': 'Connecticut',
    'DC': 'District of Columbia',
    'D.C.': 'District of Columbia',
    'D.C': 'District of Columbia',
    'DE': 'Delaware',
    'FL': 'Florida',
    'GA': 'Georgia',
    'HI': 'Hawaii',
    'IA': 'Iowa',
    'ID': 'Idaho',
    'IL': 'Illinois',
    'IN': 'Indiana',
    'KS': 'Kansas',
    'KY': 'Kentucky',
    'LA': 'Louisiana',
    'MA': 'Massachusetts',
    'MD': 'Maryland',
    'ME': 'Maine',
    'MI': 'Michigan',
    'MN': 'Minnesota',
    'MO': 'Missouri',
    'MS': 'Mississippi',
    'MT': 'Montana',
    'NC': 'North Carolina',
    'ND': 'North Dakota',
    'NE': 'Nebraska',
    'NH': 'New Hampshire',
    'NJ': 'New Jersey',
    'NM': 'New Mexico',
    'NV': 'Nevada',
    'NY': 'New York',
    'OH': 'Ohio',
    'OK': 'Oklahoma',
    'OR': 'Oregon',
    'PA': 'Pennsylvania',
    'RI': 'Rhode Island',
    'SC': 'South Carolina',
    'SD': 'South Dakota',
    'TN': 'Tennessee',
    'TX': 'Texas',
    'UT': 'Utah',
    'VA': 'Virginia',
    'VT': 'Vermont',
    'WA': 'Washington',
    'WI': 'Wisconsin',
    'WV': 'West Virginia',
    'WY': 'Wyoming'
}


In [ ]:
## Location Names====================================================
print(dataset.iloc[0]['user']['location'])
states = []
for index, tweet in dataset.iterrows():
    loc = tweet['user']['location']
    for code in state_codes:
        name = state_code_to_name[code]
        if ' {code},' in loc or loc[:3] == '{code},' or loc[-3:] == ' {code}':
            states.append(name)
            break
        elif name in loc:
            states.append(name)
            break
    else:
        states.append(None)


United States


In [ ]:
state_dist = {}

for state in states:
    if state in state_dist:
        state_dist[state] += 1
    else:
        state_dist[state] = 1

dataset['state'] = states
print(state_dist)

{None: 164126, 'New York': 2547, 'California': 2550, 'Washington': 2318, 'Virginia': 545, 'Pennsylvania': 426, 'Texas': 1246, 'Maryland': 344, 'Massachusetts': 396, 'Alaska': 98, 'Florida': 1517, 'Tennessee': 239, 'New Jersey': 449, 'Missouri': 209, 'Oklahoma': 201, 'Colorado': 543, 'Utah': 124, 'Arizona': 375, 'Oregon': 534, 'New Mexico': 160, 'Illinois': 341, 'Ohio': 578, 'Mississippi': 78, 'Indiana': 340, 'Louisiana': 132, 'Kentucky': 164, 'Minnesota': 365, 'Delaware': 80, 'Michigan': 585, 'Maine': 172, 'Georgia': 319, 'Nevada': 110, 'Kansas': 256, 'Wisconsin': 300, 'Nebraska': 78, 'South Carolina': 149, 'Idaho': 74, 'Iowa': 161, 'Vermont': 113, 'North Carolina': 434, 'Wyoming': 32, 'Connecticut': 199, 'Montana': 91, 'Rhode Island': 106, 'Arkansas': 103, 'Hawaii': 129, 'District of Columbia': 21, 'North Dakota': 16, 'Alabama': 148, 'South Dakota': 37, 'New Hampshire': 117}


In [ ]:
state_dataset = dataset[dataset['state'].notnull()]

In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:

from transformers import pipeline 
sentiment_model = pipeline(model="XerOpred/twitter-climate-sentiment-model")

sentiments = []
n = 0
tot = len(state_dataset)
for index, tweet in state_dataset.iterrows():
    v = sentiment_model(tweet['full_text'])[0]['score']
    if(v > 0.98):
        sentiments.append(1)
    else:
        sentiments.append(-1)
    n += 1
    if(n % 100 == 0):
        print(f'{n} / {tot} : {(n/tot)*100:10.2f}')
    
state_dataset['sentiment'] = sentiments


Downloading:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

100 / 20649 :       0.48
200 / 20649 :       0.97
300 / 20649 :       1.45
400 / 20649 :       1.94
500 / 20649 :       2.42
600 / 20649 :       2.91
700 / 20649 :       3.39
800 / 20649 :       3.87
900 / 20649 :       4.36
1000 / 20649 :       4.84
1100 / 20649 :       5.33
1200 / 20649 :       5.81
1300 / 20649 :       6.30
1400 / 20649 :       6.78
1500 / 20649 :       7.26
1600 / 20649 :       7.75
1700 / 20649 :       8.23
1800 / 20649 :       8.72
1900 / 20649 :       9.20
2000 / 20649 :       9.69
2100 / 20649 :      10.17
2200 / 20649 :      10.65
2300 / 20649 :      11.14
2400 / 20649 :      11.62
2500 / 20649 :      12.11
2600 / 20649 :      12.59
2700 / 20649 :      13.08
2800 / 20649 :      13.56
2900 / 20649 :      14.04
3000 / 20649 :      14.53
3100 / 20649 :      15.01
3200 / 20649 :      15.50
3300 / 20649 :      15.98
3400 / 20649 :      16.47
3500 / 20649 :      16.95
3600 / 20649 :      17.43
3700 / 20649 :      17.92
3800 / 20649 :      18.40
3900 / 20649 :      1

C:\Users\Hoppenot\AppData\Local\Temp\ipykernel_32332\1958388902.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  state_dataset['sentiment'] = sentiments


In [ ]:
state_dataset.filter(items=['state','sentiment','created_at']).to_csv('./out.csv', index=False)